In [ ]:
import numpy as np 
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
submission=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv')
test=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')

In [ ]:
def aboutdata(dataframe):
    df=dataframe
    print('About datatypes of columns and memory usage:')
    df.info()
    print('Shape of data frame:')
    print(df.shape)
    for col in df.columns:
        print("Unique number of values in ")
        print(col)
        print(df.loc[:,col].nunique())
    print("number of null values present in each column")
    print(df.isnull().sum())
    print(df.head(3))

In [ ]:
aboutdata(train)

In [ ]:
aboutdata(test)

In [ ]:
aboutdata(submission)

In [ ]:
train.drop(['date_block_num','item_price'],axis = 1 ,inplace = True)

In [ ]:
train['date']=pd.to_datetime(train['date'],dayfirst=True) 

In [ ]:
train['date']=train['date'].apply(lambda x: x.strftime("%Y-%m"))

In [ ]:
train['date']

In [ ]:
train.head()

In [ ]:
train.plot(x='date',y='item_cnt_day',figsize= (40,20))

In [ ]:
train   =   train.groupby(['date','shop_id','item_id']).sum()
train   =   train.pivot_table(index=['shop_id','item_id'],columns = 'date',values = 'item_cnt_day',fill_value = 0)
train.reset_index(inplace = True)




train.head()

In [ ]:
y_train = train['2015-10'].values
x_train = train.drop(['2015-10'],axis=1)

In [ ]:
rf  =  RandomForestClassifier(n_estimators=100,n_jobs=-1,verbose=1)
rf.fit(x_train,y_train)

In [ ]:
test = pd.merge(test, train, on=['shop_id','item_id'], how='left')
test.drop(['ID', '2013-01'], axis=1, inplace=True)
test = test.fillna(0)
test.head()

In [ ]:
predictions  =   rf.predict(test)
predictions =   list(map(round,predictions))

In [ ]:
final = pd.DataFrame({'ID':np.arange(0,len(predictions),1),'item_cnt_month':predictions})
final.head()

In [ ]:
final.to_csv('submission.csv',index=False,encoding='utf-8')

In [ ]:
final

In [ ]:
#op=max(final.item_cnt_month)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
plt.style.use("ggplot")
final.plot("ID","item_cnt_month",c = "r") 
plt.legend(labels=["item_cnt_month"], loc = 'best')
plt.xlabel("ID", fontweight = "bold") 
plt.ylabel("item_cnt_month", fontweight = "bold")
plt.title("test_prediction", fontsize = 15, fontweight = "bold", y = 1.1)   
plt.xticks(rotation=45)
plt.savefig("Motorcycles growth.jpg",
            bbox_inches='tight',
            pad_inches=0.0,)
plt